# MADA Model Implementation

This notebook imports the customized deeplabv3 model from MADA codebase. Note that `schedule`, `loss`, `optimizer`, and `model` folders are all from the MADA codebase and are written in pytorch.

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## The final customized model

Cannot be run here. Need full project setup to have cfg, writer and logger

In [3]:
from models.adaptation_model import CustomModel
# model = CustomModel(cfg, writer, logger)

## Inner level of the customized model

In [4]:
from loss import get_loss_function
from models.deeplab import DeepLab
from models.sync_batchnorm import SynchronizedBatchNorm2d, DataParallelWithCallback
from schedulers import get_scheduler

In [5]:
PredNet = DeepLab(num_classes=19, backbone='resnet101', 
                  output_stride=16,bn='sync_bn',freeze_bn=True,)

In [6]:
PredNet.eval()

DeepLab(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): SynchronizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): SynchronizedBatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=T

The original MADA code uses the following code to generate feat_cls and output
```python
for (images, labels, source_img_name) in tqdm(datasets.target_train_loader):
    _, _, feat_cls, output = PredNet(images)
    print(output.shape, feat_cls.shape)
```

In [22]:
from torchsummary import summary
# summary(PredNet, (3, 1052, 1914)), no kernel image is on the device